<a href="https://colab.research.google.com/github/almutareb/InnovationPathfinderAI/blob/agent_notebook/mixtral_ollama_public_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://github.com/jmorganca/ollama/blob/main/examples/jupyter-notebook/ollama.ipynb

In [ ]:
!curl https://ollama.ai/install.sh | sh
!command -v systemctl >/dev/null && sudo systemctl stop ollama

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8423    0  8423    0     0   8293      0 --:--:--  0:00:01 --:--:--  8298>>> Downloading ollama...
100  8423    0  8423    0     0   7756      0 --:--:--  0:00:01 --:--:--  7763
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 0.0.0.0:11434.
>>> Install complete. Run "ollama" from the command line.
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down


In [ ]:
!pip install aiohttp pyngrok -q

In [ ]:
from pyngrok import ngrok
from google.colab import userdata

# https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_TOKEN = userdata.get('NGROK_TOKEN')
ngrok.set_auth_token("NGROK_TOKEN")

In [ ]:


import os
import asyncio
from aiohttp import ClientSession
from google.colab import userdata

# Set LD_LIBRARY_PATH so the system NVIDIA library becomes preferred
# over the built-in library. This is particularly important for
# Google Colab which installs older drivers
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})
NGROK_TOKEN = userdata.get('NGROK_TOKEN')
ngrok.set_auth_token("NGROK_TOKEN")

async def run(cmd):
  '''
  run is a helper function to run subcommands asynchronously.
  '''
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )


await asyncio.gather(
    run(['ollama', 'serve']),
    run(['ngrok', 'config', 'add-authtoken', NGROK_TOKEN]),
    run(['ngrok', 'http', '--log', 'stderr', '11434']),
    # run(["ollama","run","llama2"]),
    run(["ollama","run","mistral"]),
)